# Estimació p* i q* a partir d'una latent amb probabilitat uniforme en cas d'empat

Objectius del notebook:
- Definir una Latent L* on el cas d'empat (1-0 o 0-1) es decideixi amb una probabilitat 50/50, en comptes d'escollir 0 directament com en el notebook previ.
- Inferir nous errors p* (probabilitat de crear un enllaç) i q* (probailitat d'eliminar-ne un) a partir de L* i les observacions de parells de xarxes A_test.
- Generar parells de xarxes a partir de L*, p* i q* i calcular-ne les energies.
- Representar en un historigrama.
- Més endavant, comparar per diferents valors de p i q.

### Previ

In [3]:
import itertools
import pickle
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import os
import time
from numba import jit, njit
from numba.types import bool_, int_, float32
from math import comb
from copy import deepcopy
from tqdm import tqdm
import networkx as nx
import pandas as pd
from collections import defaultdict
from itertools import permutations

#### Carreguem dades:

In [4]:
L = np.load("A2_blueprint.npy")
Nx=L.shape[0]
Ny=L.shape[1]

#### Generacio de xarxes amb els errors p_paper i q_paper

In [5]:
#variables d'interes

q_paper = 0.15
p_paper = 0.007
num_networks = 2
#defineixo funcio CreateCopies
def CreateCopies(L,q,p,num_networks):
    Nx = len(L) #nombre de nodes 
    A_test = np.zeros((num_networks, Nx, Nx)) #crea un array 3D ple de zeros

    p = p_paper
    q = q_paper

    for k in range(num_networks):
    #    A_copy = L.copy() #deep copy de L, però no comparteixen memòria. modificar a.copy no modifica L
        for i in range(Nx):
            for j in range(Nx):
                if L[i,j] == 1:
                    # enllaç existent -> pot ser esborrat
                    if np.random.rand() >= q: #genero aleatori entre 0 i 1
                        A_test[k,i,j] = 1
                else:
                    # posició buida -> pot crear-se un fals
                    if np.random.rand() < p:
                        A_test[k,i,j] = 1
        #A_test[k,:,:] = A_copy #guardo cada xarxa k generada.
        # per cada k (ie per cada xarxa), es comença de zero fent una còpia nova i completa de L.


    #print("Xarxes generades amb p i q del paper:")
    print(A_test.shape)
    np.save("synthetic_paper.npy", A_test)
    return A_test


In [6]:
np.random.seed(11) #hem tret la llavor fixa per tenir variabilitat
for nrep in range(10):
    A_test = CreateCopies(L, q_paper, p_paper, num_networks=2)
    #print('rep',nrep,'xarxa 1',np.sum(np.sum(A_test[0]-L)))
    #print('rep',nrep,'xarxa 2',np.sum(np.sum(A_test[1]-L)))


(2, 224, 224)
(2, 224, 224)
(2, 224, 224)
(2, 224, 224)
(2, 224, 224)
(2, 224, 224)
(2, 224, 224)
(2, 224, 224)
(2, 224, 224)
(2, 224, 224)


#### Funcions d'energia del repo

In [7]:
n_groups = 1 #considerem que els nodes pertanyen a un únic grup
start_groups = np.zeros(1) #indica on comença cada grup
end_groups = np.zeros(1) + Nx #indica on acaba
size_groups = np.zeros(1) + Nx

start_groups, end_groups, size_groups = start_groups.astype(int), end_groups.astype(int), size_groups.astype(int)

In [8]:
@jit(nopython = True)
def hamiltonian_prob(Edges_NoL, Edges_L, overlap_0, overlap_1, alpha, beta):

    A_1 = overlap_1 + alpha #o11 al paper
    B_1 = (Edges_L - overlap_1 + beta) #o10 al paper
    C_1 = Edges_L + alpha + beta #o1 al paper
    
    A_0 = overlap_0 + alpha #o00 al paper
    B_0 = (Edges_NoL - overlap_0 + beta) # o01 al paper
    C_0 = Edges_NoL + alpha + beta # o0 al paper
    
    #  [ math.lgamma(n+1) == log(n!) ]
    H1 = math.lgamma(A_1)+ math.lgamma(B_1) - math.lgamma(C_1) 
    H0 = math.lgamma(A_0)+ math.lgamma(B_0) - math.lgamma(C_0) 
    
    H = -(H1 + H0)
    return H
    
    
@jit(nopython=True)
def overlap_total_prob(L_f, A_f, P_inv_f):
    Nx = L_f.shape[0]
    Ny = L_f.shape[1]
    K = A_f.shape[0]
    
    ovlp_0 = np.zeros((K))
    ovlp_1 = np.zeros((K))
    for k in range(0,K):
        for f in range(0,Ny): 
            for c in range(0,Nx):
                p_f=int(P_inv_f[k,f]) #es tradueix l'index del bliueprint al real de 
                 #(per exemple, si el node 1 de L es el node 5 de A, p_f=5)
                p_c=int(P_inv_f[k,c])  
                valor_L, valor_A = L_f[f,c], A_f[k,p_f,p_c] #compara el valor de L i A a la posició corresponent
                
                ovlp_0[k] = ovlp_0[k] + (1-valor_L)*(1-valor_A )#mira si els zeros coincideixen a L i a A
                ovlp_1[k] = ovlp_1[k] + valor_L*valor_A #mira si els 1s coincideixen a 0 i a  A
                
                
    ovlp_1 = int(sum(ovlp_1))
    ovlp_0 = int(sum(ovlp_0)) #se sumen coincidencies en totes les observacions, resultat global
    return ovlp_0, ovlp_1
#retorna ovlp_0 (nre total de no-enllaços ben predits) i ovlp_1 (nre total d'enllacos ben predits)

@jit(nopython=True) # The blueprint is the average of the observations (taking into account the mapping)  
def L_wiring(A_f, P_inv_f): #enlaces que aparecen en la mayoría de observaciones
    #A_f:  te la forma KxNxNy, on K és el nre de xarxes observades
    Nx = A_f.shape[1]
    Ny = A_f.shape[2]
    K = A_f.shape[0] 
    L_new_f = np.zeros((Nx,Ny))
    
    for i in range(0,Nx):
        for j in range(0,Ny):
            for k in range(0,K):
                #per cada posicio (ij) mira en les K xarxes la posicio que li correspon al blueprint (equivalent segons el mapping)
                p1 = int(P_inv_f[k,i]) # Mapping of the observations
                p2 = int(P_inv_f[k,j]) # Mapping of the observations
                L_new_f[i,j] += A_f[k,p1,p2] #sumo les connexions existents a la posició equivalent en les K xarxes observades
            valor_lnew=1/K* L_new_f[i,j] #fa la mitjana de connexions existents en les K xarxes observades
            L_new_f[i,j] = round( valor_lnew ) #redondeja al valor més proper
            # If valor_lnew = 0, L=0 (we could establish L=1, but it is more probable to not have a connection)
            #L_ij=1--> a la majoria de les k observacions alineades hi ha enllaç aquí
            # L_ij=0--> a la majoria de les k observacions alineades no hi ha enllaç aquí 
    return L_new_f

## Definim L_wiring_nova

In [9]:
#volem fer un wiring que, en cas d'empat, assigni amb probabilitat un 1 o un 0 en comptes de redondejar al valor més proper. 
#poso r=0.5 per assignar amb igual probabilitat un 1 o un 0, però es pot modificar per assignar amb més probabilitat un 1 o un 0.
def L_wiring_nova(A_f, P_inv_f, r=0.5): 
    #A_f:  te la forma KxNxNy, on K és el nre de xarxes observades
    Nx = A_f.shape[1]
    Ny = A_f.shape[2]
    K = A_f.shape[0] 
    L_new_f = np.zeros((Nx,Ny))
    
   
    for i in range(0,Nx):
        for j in range(0,Ny):
            suma = 0
            for k in range(0,K):
                #per cada posicio (ij) mira en les K xarxes la posicio que li correspon al blueprint (equivalent segons el mapping)
                p1 = int(P_inv_f[k,i]) # Mapping of the observations
                p2 = int(P_inv_f[k,j]) # Mapping of the observations
                suma += A_f[k,p1,p2] #sumo les connexions existents a la posició equivalent en les K xarxes observades
            valor_lnew = suma/K #fa la mitjana de connexions existents en les K xarxes observades
            
            if suma == K/2: #si les dues sumen 1 (tenim una amb 0 i una amb 1)
                if np.random.rand() < r:
                    L_new_f[i,j] = 1
                else:
                    L_new_f[i,j] = 0
            else:
                L_new_f[i,j] = round(valor_lnew) #si no hi ha empat, valor_lnew=0 o valor_lnew=1
    return L_new_f

In [10]:
A_test = CreateCopies(L, q_paper, p_paper, num_networks=2)

Nx = A_test.shape[1]

Ao_enters = A_test.astype(int)
P_inv_id = np.tile(np.arange(Nx), (A_test.shape[0], 1))

L_star = L_wiring_nova(Ao_enters, P_inv_id)

(2, 224, 224)


In [11]:
zeros_wiring = 0
uns_wiring = 0

nre_p = 0
nre_q = 0

Nx = L_star.shape[0]

for i in range(Nx):
    for j in range(Nx):

        if L_star[i, j] == 0:
            zeros_wiring += 1

            if A_test[0, i, j] == 1:
                nre_p += 1
            if A_test[1, i, j] == 1:
                nre_p += 1

        else:
            uns_wiring += 1

            if A_test[0, i, j] == 0:
                nre_q += 1
            if A_test[1, i, j] == 0:
                nre_q += 1

p_inf = nre_p / (2 * zeros_wiring)
q_inf = nre_q / (2 * uns_wiring)

print("p_inf:", p_inf)
print("q_inf:", q_inf)

p_inf: 0.005878306547432931
q_inf: 0.1398093508851566
